## Get Suspicious DNS

In [ ]:
import urllib2
import json
import os

# getting date from the parent path.
path = os.getcwd().split("/")
date = path[len(path)-1] 

spath = os.environ['LUSER'] + '/ipython/dns/user/' + date + '/'
sconnect = spath + 'dns_scores.csv'
sconnectbu = spath + 'score_tmp.csv'
ips = []
dns = []
with open(sconnect, 'r') as f:
    reader = csv.reader(f, delimiter=',')
    reader.next()
    for row in reader: 
        # frame_time, frame_len, ip_dst, dns_qry_name, dns_qry_class,  dns_qry_type, dns_qry_rcode, domain, subdomain, 
        #      0    ,      1   ,    2  ,       3     ,          4   ,         5    ,        6     ,   7   ,    8     ,
        #subdomain_length, num_periods, subdomain_entropy, top_domain, word, score, query_rep,  hh, 
        #      9          ,      10    ,     11          ,    12     ,  13 ,  14  ,   15     ,  16, 
        #ip_sev, dns_sev, dns_qry_class_name, dns_qry_type_name, dns_qry_rcode_name, network_context
        # 17   ,   18   ,           19      ,      20          ,         21        ,      22    
        if row[2] not in ips and row[17] == '0': 
             ips.append(row[2])
        if row[3] not in dns and row[18] == '0':
            dns.append(row[3]) 
        

In [ ]:
import ipywidgets as widgets
# from IPython.html import widgets
# from IPython.html.widgets import interact, interactive, fixed
from IPython.display import display, HTML, clear_output, Javascript 

def fill_list(list_control,source):
    options_list = ['--Select--'] 
    options_list.extend([s for s in source])
    list_control.options = options_list

# client panel
client_header = widgets.HTML(value="Client IP")
client_select = widgets.Select(height=190, width=120)  
fill_list(client_select,ips)
client_select.value = "--Select--"
client_box = widgets.Box(width=150, height=200)
client_box.children = [client_header, client_select]

# query panel
query_header = widgets.HTML(value="Query")
query_select = widgets.Select(height=190, width=250) 
fill_list(query_select,dns)

query_box = widgets.Box(width=300, height=210)
query_box.children = [query_header, query_select]

# Actions Panel
actions_header = widgets.HTML(value="&nbsp;")
quick_text = widgets.Text(value='',width=150, placeholder='Quick scoring')
rating_btn = widgets.RadioButtons(description='Rating:', options=['1', '2', '3'], width=90)
assign_btn = widgets.Button(description='Score')
save_btn = widgets.Button(description='Save')
actions_box = widgets.Box(width=300, height=210)
actions_box.children = [actions_header,quick_text,rating_btn, assign_btn,save_btn]

scoring_form = widgets.HBox(width=600, height=200)
scoring_form.children = [client_box,query_box,actions_box]

display(Javascript("$('.widget-area > .widget-subarea > *').remove();"))
display(scoring_form)


# Update Suspicious DNS

In [ ]:
import csv
import datetime
import subprocess 

def assign_score(b):
    score_values = []
    ip_sev = int(rating_btn.selected_label) if not "--Select--" in client_select.value else ""
    dns_sev = int(rating_btn.selected_label) if not "--Select--" in query_select.value else ""    
    mod_date = datetime.datetime.now()
    mod_user = "IPython"  
    
    if quick_text.value: 
        ip = ""
        dns = quick_text.value
        dns_sev = int(rating_btn.selected_label) 
        # Loop over every element in query_select widget
        score_values = []
        for query in query_select.options:
            if query.endswith(dns):
                # Matching element, create one row
                score_values.append((ip,query,ip_sev,dns_sev,mod_date,mod_user))
    else: 
        ip = client_select.value if not "--Select--" in client_select.value else ""
        dns = query_select.value if not "--Select--" in query_select.value else ""
        score_values.append((ip,dns,ip_sev,dns_sev,mod_date,mod_user))
        
    csv_type = 'a' if os.path.isfile("score_tmp.csv") else 'w'
    with open(sconnect, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        reader.next()
        with open(sconnectbu,csv_type) as score:
            wr = csv.writer(score, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            wr.writerow(['frame_time'] + ['frame_len'] + ['ip_dst'] + ['dns_qry_name'] + ['dns_qry_class'] + 
                        ['dns_qry_type'] + ['dns_qry_rcode']+['domain']+
                         ['subdomain']+['subdomain_length']+
                         ['num_periods']+['subdomain_entropy'] + ['top_domain']+['word']+
                         ['score']+['query_rep']+['hh']+['ip_sev']+['dns_sev']+ ['dns_qry_class_name'] + 
                        ['dns_qry_type_name'] + ['dns_qry_rcode_name'] + ['network_context'])  
            for row in reader:  
                #row[
                # frame_time, frame_len, ip_dst, dns_qry_name, dns_qry_class,  dns_qry_type, dns_qry_rcode, domain, subdomain, 
                #      0    ,      1   ,    2  ,       3     ,          4   ,         5    ,        6     ,   7   ,    8     ,
                #subdomain_length, num_periods, subdomain_entropy, top_domain, word, score, query_rep,  hh, 
                #      9          ,      10    ,     11          ,    12     ,  13 ,  14  ,   15     ,  16, 
                #ip_sev, dns_sev, dns_qry_class_name, dns_qry_type_name, dns_qry_rcode_name, network_context
                # 17   ,   18   ,           19      ,      20          ,         21        ,      22                    
                for value in score_values: 
                    if row[2] == value[0]:  
                        row[17] = value[2]  
                        break
                    if row[3] == value[1]:  
                        row[18] = value[3]   
                        break
                wr.writerow(row)        
    !cp $sconnectbu $sconnect
    !rm -f $sconnectbu 
    
    if ip != "--Select--":
        display(Javascript("$(\"option[data-value='" + ip +"']\").remove();"))

    if quick_text.value:
        display(Javascript("$(\"option[data-value$='" + quick_text.value +"']\").remove();"))
    elif dns != "--Select--":
        display(Javascript("$(\"option[data-value='" + dns +"']\").remove();"))

    client_select.value = "--Select--"
    query_select.value = "--Select--"
    quick_text.value = ""

def save(b):  
    display(Javascript('window.parent.iframeReloadHook && window.parent.iframeReloadHook();'))
        
assign_btn.on_click(assign_score)
save_btn.on_click(save)